In [4]:
import sys
import re
import os
import numpy as np
import pandas as pd
import pysam
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from Bio.Seq import Seq
import gget
import mappy as mp

In [5]:
root_dir = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/fastq/p2/"

res = []

def read_fastq(fpath):
    """a function to parse a fastq file """
    res = []
    for read in pysam.FastxFile(fpath):
        NlaIII = read.sequence.count('CATG')
        quals = read.get_quality_array()
        row = {
            'read_name' : read.name,
            'seq_length' : len(read.sequence),
            'n_NlaIII' : NlaIII,
            'mean_base_quality' : np.mean(quals),
            'median_base_quality' : np.median(quals),
            'min_base_quality' : np.min(quals),
            'max_base_quality' : np.max(quals),
            'sequence' : read.sequence,
        }

        res.append(row)
    return pd.DataFrame(res)
    
res = []
for run in os.listdir(root_dir):

    subdir_path = f"{root_dir}{run}/merged_fastq/"
    sample_files = os.listdir(subdir_path)
    sample_files = [x for x in sample_files if not x == 'unclassified.fastq']
    print(len(sample_files))

    for sample in sample_files:
        sample_path = f"{subdir_path}{sample}"
        bc_number = sample.replace("barcode", "").replace(".fastq", "")
        bc_id = f"BC{bc_number}"

        df = read_fastq(sample_path)
        df['run'] = run
        df['barcode'] = bc_id
        res.append(df)

res = pd.concat(res)
print(f"{res.shape=}")
print(f"{res['read_name'].nunique()=}")
res.head()

96


KeyboardInterrupt: 

In [ ]:
break

In [ ]:
# load ont_barcodes
fpath = "../resources/ONT_barcode_96_sequences.csv"
df = pd.read_csv(fpath)
codes = dict(zip(df.Barcode.values, df.Sequence.values))
df.head()

In [ ]:
# for _, record in res.sample(40).iterrows():
#     barcode = record['barcode']
#     read_name = record['read_name']
#     ont_seq = codes[barcode]
#     read_seq = record['sequence']
#     seq_len = record["seq_length"]
#     n_NlaIII = record["n_NlaIII"]

#     rev_comp = str(Seq(barcode).reverse_complement())

#     has_code = ont_seq in read_seq
#     has_code_rev = rev_comp in read_seq
#     print(f"{barcode=} {n_NlaIII=} {seq_len=} {has_code=}")

In [ ]:
df = []

for _, record in res.iterrows():
    barcode = record['barcode']
    read_name = record['read_name']
    ont_seq = codes[barcode]
    read_seq = record['sequence']
    seq_len = record["seq_length"]
    n_NlaIII = record["n_NlaIII"]

    hits = read_seq.count(ont_seq)
    hits_rc = read_seq.count(str(Seq(ont_seq).reverse_complement()))

    row = {
        'run' : record['run'],
        'read_name' : read_name,
        'barcode' : barcode,
        'seq_len' : seq_len,
        'n_NlaIII' : n_NlaIII,
        'n_barcodes' : hits,
        'n_barcodes_rc' : hits_rc,
        'mean_base_quality' : record['mean_base_quality'],
    }

    df.append(row)

df  = pd.DataFrame(df)
df.head()


# count NlaIII sites

In [ ]:
df['ord'] = np.where(df['n_NlaIII'] == 0, 'none', 'higher')
df['ord'] = np.where(df['n_NlaIII'] == 1, 'pair', df['ord'] )
df['ord'].value_counts(normalize=True)

# count barcodes

In [ ]:
df['cat'] = np.where(df['n_barcodes'] == 1, 'one', 'none')
df['cat'] = np.where(df['n_barcodes'] > 1, 'mulitple', df['cat'])

print(df.shape)
print(df['barcode'].nunique())
df['cat'].value_counts(normalize=True)

In [ ]:
df['cat_rc'] = np.where(df['n_barcodes_rc'] == 1, 'one', 'none')
df['cat_rc'] = np.where(df['n_barcodes_rc'] > 1, 'mulitple', df['cat_rc'])

print(df.shape)
print(df['barcode'].nunique())
df['cat_rc'].value_counts(normalize=True)

In [ ]:
plt.rcParams['figure.figsize'] = 5, 3
plt.rcParams['figure.dpi'] = 200

M = pd.crosstab(df['cat'], df['ord'], normalize=True)
print(M)
sns.heatmap(M, annot=True,  fmt=".2f", lw=1, cbar=False)

In [ ]:
plt.rcParams['figure.figsize'] = 5, 3
plt.rcParams['figure.dpi'] = 200

M = pd.crosstab(df['cat_rc'], df['ord'], normalize=True)
print(M)
sns.heatmap(M, annot=True,  fmt=".2f", lw=1, cbar=False)

In [ ]:
plt.rcParams['figure.figsize'] = 5, 3
plt.rcParams['figure.dpi'] = 200

M = pd.crosstab(df['cat'], df['cat_rc'], normalize=True)
print(M)
sns.heatmap(M, annot=True,  fmt=".2f", lw=1, cbar=False)

# number of reads with exactly one barcode (RC + forward)

In [ ]:
test = df[df['n_barcodes'].isin([0, 1])]
test = test[test['n_barcodes_rc'].isin([0, 1])]

test = test[(test['n_barcodes'] != 0) & (test['n_barcodes_rc'] != 0)]

print(f"{test.shape=}")
print(test['ord'].value_counts(normalize=True))

test.head()

In [ ]:
plt.rcParams['figure.figsize'] = 8, 2.5
plt.rcParams['figure.dpi'] = 200

sns.boxplot(data=df, 
                x='n_barcodes',
                y='mean_base_quality',
                showfliers=False)

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 8, 2.5
plt.rcParams['figure.dpi'] = 200

sns.boxplot(data=df, 
                x='n_barcodes',
                y='seq_len',
                showfliers=False)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 4, 3
plt.rcParams['figure.dpi'] = 200

sns.boxplot(data=df, 
                x='run',
                y='n_barcodes')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = 8, 2.5
plt.rcParams['figure.dpi'] = 200

sns.boxplot(data=df,
            x='barcode',
            y='n_barcodes',
            showfliers=False)

plt.xticks([])
plt.show()

# multiple barcodes

In [ ]:
# let's look at a few extreme cases
barcodes = df[df['n_barcodes'] > 15]['read_name'].unique()
pdf = res[res['read_name'].isin(barcodes)]

for _, record in pdf.head(10).iterrows():
    print("===== ", record['run'], record['barcode'], " =====")
    ont_seq = codes[record['barcode']]
    ont_seq_rc = str(Seq(ont_seq).reverse_complement())
    print('READ NAME: ', record['read_name'])
    print('NUMBER NlaIII SITES: ', record['n_NlaIII'])
    print('ONT BARCODE: ', ont_seq)
    print('ONT BARCODE (RC): ', ont_seq_rc)

    red_ont = f"\033[91m{ont_seq}\033[0m"
    blue_ont = f"\033[94m{ont_seq_rc}\033[0m"
    cutter = f"\033[92mCATG\033[0m"
    highlighted_text = re.sub(ont_seq, red_ont, record['sequence'])
    highlighted_text = re.sub(ont_seq_rc, blue_ont, highlighted_text)
    highlighted_text = re.sub('CATG', cutter, highlighted_text)
    print('READ SEQUENCE: ', highlighted_text)
    print()

# Extactly one forward barcode

In [ ]:
# let's look at a few extreme cases
barcodes = df[(df['n_barcodes'] == 1) ]['read_name'].unique()
pdf = res[res['read_name'].isin(barcodes)]


for _, record in pdf.head(10).iterrows():
    print("===== ", record['run'], record['barcode'], " =====")
    ont_seq = codes[record['barcode']]
    ont_seq_rc = str(Seq(ont_seq).reverse_complement())
    print('READ NAME: ', record['read_name'])
    print('NUMBER NlaIII SITES: ', record['n_NlaIII'])
    print('ONT BARCODE: ', ont_seq)
    print('ONT BARCODE (RC): ', ont_seq_rc)

    red_ont = f"\033[91m{ont_seq}\033[0m"
    blue_ont = f"\033[94m{ont_seq_rc}\033[0m"
    cutter = f"\033[92mCATG\033[0m"
    highlighted_text = re.sub(ont_seq, red_ont, record['sequence'])
    highlighted_text = re.sub(ont_seq_rc, blue_ont, highlighted_text)
    highlighted_text = re.sub('CATG', cutter, highlighted_text)
    print('READ SEQUENCE: ', highlighted_text)
    print()


In [ ]:
# let's look at a few extreme cases
barcodes = df[(df['n_barcodes_rc'] == 1) ]['read_name'].unique()
pdf = res[res['read_name'].isin(barcodes)]


for _, record in pdf.head(10).iterrows():
    print("===== ", record['run'], record['barcode'], " =====")
    ont_seq = codes[record['barcode']]
    ont_seq_rc = str(Seq(ont_seq).reverse_complement())
    print('READ NAME: ', record['read_name'])
    print('NUMBER NlaIII SITES: ', record['n_NlaIII'])
    print('ONT BARCODE: ', ont_seq)
    print('ONT BARCODE (RC): ', ont_seq_rc)

    red_ont = f"\033[91m{ont_seq}\033[0m"
    blue_ont = f"\033[94m{ont_seq_rc}\033[0m"
    cutter = f"\033[92mCATG\033[0m"
    highlighted_text = re.sub(ont_seq, red_ont, record['sequence'])
    highlighted_text = re.sub(ont_seq_rc, blue_ont, highlighted_text)
    highlighted_text = re.sub('CATG', cutter, highlighted_text)
    print('READ SEQUENCE: ', highlighted_text)
    print()

# No forward barcodes, reverse only

In [ ]:
# let's look at a few extreme cases
barcodes = df[(df['n_barcodes'] == 0) & (df['n_barcodes_rc'] == 1) ]['read_name'].unique()
pdf = res[res['read_name'].isin(barcodes)]


for _, record in pdf.head(10).iterrows():
    print("===== ", record['run'], record['barcode'], " =====")
    ont_seq = codes[record['barcode']]
    ont_seq_rc = str(Seq(ont_seq).reverse_complement())
    print('READ NAME: ', record['read_name'])
    print('NUMBER NlaIII SITES: ', record['n_NlaIII'])
    print('ONT BARCODE: ', ont_seq)
    print('ONT BARCODE (RC): ', ont_seq_rc)

    red_ont = f"\033[91m{ont_seq}\033[0m"
    blue_ont = f"\033[94m{ont_seq_rc}\033[0m"
    cutter = f"\033[92mCATG\033[0m"
    highlighted_text = re.sub(ont_seq, red_ont, record['sequence'])
    highlighted_text = re.sub(ont_seq_rc, blue_ont, highlighted_text)
    highlighted_text = re.sub('CATG', cutter, highlighted_text)
    print('READ SEQUENCE: ', highlighted_text)
    print()


In [ ]:
break

In [ ]:
# look for all barcodes in a few sequences
for _, record in res.sample(100).iterrows():
    read_seq = record['sequence']
    barcode = record['barcode']
    for bc, ont_seq in codes.items():
        if ont_seq in read_seq:
            print(f"cell barcode: {barcode} found: {bc}")